# API Request for Connection to Sentinel Hub <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:**
[S1GRD](http://www.esa.int/Applications/Observing_the_Earth/Copernicus/Sentinel-1/Data_products)

*Dataset is external to the Digital Earth Africa platform.*


## Description

This notebook demonstrates the following steps:
1. Introduction to Sentinel Hub and its API Data and services for Sentinel-1  
2. Estimate two-month temporal average on S1GRD orthorectified Gamma0 in VV polarization over part of Rome, Italy

***

## Introduction to Sentinel Hub

[Sentinel Hub](https://www.sentinel-hub.com/explore) is a multi-spectral and multi-temporal big data satellite imagery service, capable of fully automated archiving, real-time processing and distribution of remote sensing data and related Earth Observation products. It is an engine for processing of petabytes of satellite data that opens the door for machine learning and helping hundreds of application developers worldwide. It makes Sentinel, Landsat, and other Earth observation imagery easily accessible for browsing, visualization and analysis. Sentinel Hub is operated by [Sinergise](https://www.sinergise.com/), a company that have developed enterprise level spatial solutions for governmental clients in Europe and Africa.

> Users can access Sentinel Hub's Application Program Interface (API) to retrieve satellite data over their Area of Interest (AOI) and specific time range from full archives in a matter of seconds.

## Sentinel-1AB Collection 

The [Sentinel-1](https://sentinel.esa.int/web/sentinel/missions/sentinel-1) imagery is provided by two polar-orbiting satellites, operating day and night performing C-band synthetic aperture radar imaging, enabling them to acquire imagery regardless of the weather. Main applications are for monitoring sea ice, oil spills, marine winds, waves & currents, land-use change, land deformation, and to respond to emergencies such as floods and earthquakes. Sentinel Hub currently supports Sentinel-1 Level-1 GRD (Ground Range Detected) products only.  

## How to access Sentinel Hub data and services

The Sentinel Hub API is a RESTful API interface to various satellite [imagery archives](https://docs.sentinel-hub.com/api/latest/#/data/). It provides access to raw satellite data, rendered images, statistical analysis and much more. In order to access and use sentinel hub services through API calls, the user needs to `follow these three steps`:

1. Register to  [Sentinel Hub](https://www.sentinel-hub.com/) portal 
2. [Authentication](https://docs.sentinel-hub.com/api/latest/#/API/authentication)
3. [Process](https://docs.sentinel-hub.com/api/latest/#/Evalscript/) 

Following registration to the Sentinel Hub, the user will be allocated a non-commercial account with dynamic dashboard that shows the subscription plan, processing units and the number of requests per minute.

## Data Availibility 

Sentinel Hub Earth Observation collection is available in several deployments including:
- Main Sentinel Hub deployment - EU-1-Central region 
- Main Sentinel Hub deployment - US-West-2 region
- CreoDIAS Sentinel Hub deployment
- EOCloud Sentinel Hub deployment

The hub can be searched and accessed through WMS services following registration to the Sentinel hub.  

> Enter the `WMS URL endpoint` location for global Earth Observation Dataset using configuration dashboard.  

In [50]:
#enter the WMS URL endpoint for specific EO collection 
URL = 'https://services.sentinel-hub.com/ogc/wms/7677773d-8b8c-4ba8-b7ee-a10316f65ada'

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load Packages" cell. 

## Load Packages

Import python packages that are used for API authentication and request. 
> Using `pip install owslib` users can install the package locally followed by restarting the kernel. 

In [51]:
import owslib
import os 
import uuid
import pandas as pd 
import xarray as xr 
import subprocess 
from owslib.wms import WebMapService
from IPython.display import display

## Explore the WMS service 
Function to Explore WMS service available for specific data collection on Sentinel hub portal 

In [52]:
def wms_layers_styles(url):
    # Get WMS data from URL
    wms_items = dict(WebMapService(url, version='1.3.0').items())
    layers_styles = {key: val.styles.keys() for key, val in wms_items.items()}
    # Create a dataframe with an index of layer names and list of styles
    layers_df = pd.DataFrame(index=layers_styles.keys())
    layers_df['styles'] = layers_styles.values()
    layers_df.index.name = 'layer'
    # Print output
    with pd.option_context('display.max_rows', None, 
                           'display.max_colwidth', 255):
        display(layers_df)
    return layers_df

In [53]:
layers_df = wms_layers_styles(url=URL)

,styles
layer,
TRUE-COLOR-S2-L2A,(default)
BANDS-MODIS,(default)
TRUE-COLOR-S1-IW,(default)
TRUE-COLOR-S2-L1C,(default)
BANDS-L8,(default)
BANDS-S1-EW-SH,(default)
BANDS-S1-IW,(default)
BANDS-S2-L2A,(default)
BANDS-S2CLOUDLESS,(default)


Function to explore the data by querrying for specific area 

In [54]:
def wms_xarray(url, 
               layer, 
               bbox, 
               layerstyle, 
               layertime,                
               layerformat='png', 
               layersizex=512, 
               layersizey=512,
               epsg='EPSG:4326'):
    # Get extent
    y_max, x_min, y_min, x_max = bbox
    # Use unique ID to solve issue where xr.open_rasterio
    # caches the output result
    unique_id = str(uuid.uuid4())
    infile = f'{unique_id}.tif'
    outfile = f'{unique_id}_georef.tif'
    wms = WebMapService(url, version='1.3.0')
    crs = sorted(wms[layer].crsOptions)
    time = wms[layer].timepositions
    output = wms.getmap(layers=[layer],
                styles=[layerstyle],
                srs='EPSG:4326',
                bbox=(x_min, y_min, x_max, y_max),
                size=(layersizex, layersizey),
                format='image/'+layerformat,
                time=layertime
                )
    with open(infile, 'wb') as out:
        out.write(output.read())   
    # Assign projection data (to improve)
    subprocess.call(['gdal_translate','-a_srs', epsg, 
                     '-a_ullr', str(x_min), str(y_max), str(x_max), str(y_min), 
                     infile, outfile])
    ds = xr.open_rasterio(outfile)
    # Remove temporary rasters
    os.remove(outfile)
    os.remove(infile )
    return(ds)

## Analysis Parameters

For a full descriptions of options for loading `Sentinel-1 GRD` data see the [product description](https://docs.sentinel-hub.com/api/latest/#/data/Sentinel-1-GRD)

The following default parameters will load Sentinel 1-GRD Orthorectified Gamma0 in VV polarization data over SOME LOCATION IN AFRICA HERE.

You can change the parameters below to load a different location and diffent time period.

In [33]:
AOI = 
time_start =
time_end =
mosaicking = 
bands = 
tiff_name = 'TemporalAverage_S1_VV2.tif'

In [60]:
WebMapService(url=URL, version='1.3.0').items()
WebMapService(url=URL, version='1.3.0')["BANDS-L8"].styles

{'default': {'title': 'default'}}

### Generate an evalscript

This is how we query and extract data from the API

In [61]:
ds = wms_xarray(url=URL,  #WMS for background image
                layer="BANDS-L8",  #WMS layer name
                bbox= [-28.157021, 140.961682, -29.159082, 141.109154], 
                layerstyle='default',
                layertime='2020-04-02T23:59:59Z')

ServiceException: <?xml version='1.0' encoding="UTF-8"?>
<ServiceExceptionReport version="1.3.0"
	xmlns="http://www.opengis.net/ogc"
	xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
	xsi:schemaLocation="http://www.opengis.net/ogc http://schemas.opengis.net/wms/1.3.0/exceptions_1_3_0.xsd">
	<ServiceException>
		<![CDATA[ Dataset type L8L1C not supported here. Please use https://services-uswest2.sentinel-hub.com endpoint. ]]>
	</ServiceException>
</ServiceExceptionReport>

In [47]:
print(type(ds))

<class 'xarray.core.dataarray.DataArray'>


In [48]:
S1GRD = xr.open_rasterio(ds)
S1GRD.plot(figsize=(10,10))

TypeError: unhashable type: 'DataArray'